<a href="https://colab.research.google.com/github/royam0820/crewai/blob/main/L2_research_write_article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

DeepLearning Course on [Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/4/create-agents-to-research-and-write-an-article-(code))

# Writing a blog post

This Jupyter notebook is designed to **automate the process of writing a blog post on a given topic using agent-based AI systems**. It utilizes the crewai library to create and manage AI agents with different roles in the content creation workflow.

The notebook defines three main agents—
- Content Planner,
- Content Writer, and
- Editor, and
- orchestrates their tasks using the Crew class to produce a well-written blog post.

You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`



```
crewai==0.28.8
crewai_tools==0.1.6
langchain_community==0.0.29# This is formatted as code
```



## Library Installation and Imports

In [1]:
!pip install crewai==0.28.8
!pip install crewai_tools--0.1.6
!pip install langchain_community==0.0.29 # this is formatted as code
!pip install utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 781.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

NB: Suppresses any warning messages to ensure a clean output.

## importing required classes

In [3]:
from crewai import Agent, Task, Crew
import os
from google.colab import userdata


- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

## Setting up API Keys

In [4]:
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'


## Defining Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Content Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [5]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	  verbose=True
)

NB: The Content Planner agent is responsible for creating a comprehensive plan for the blog article on the given topic.

### Content Writer

In [6]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

NB: The Content Writer agent uses the plan from the Content Planner to write the blog post, ensuring it is insightful and accurate.

### Editor

In [7]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

The Editor agent reviews the content for grammatical correctness and brand consistency.

## Defining Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

NB: This task is associated to the Content Planner agent and involves creating a detailed content plan, including an outline, key points, audience analysis, and SEO suggestions.

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

NB: This task is assigned to the content writer agen. The Write task leverages the content plan to write the blog post, ensuring it follows the provided structure and SEO guidelines.

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

NB: This task is assiged to the Editor agent. The Edit task involves proofreading and finalizing the blog post for publication.

## Assembling the Crew and Executing the Workflow

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution.

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

NB: **The Crew class organizes the agents and tasks into a coherent workflow.**

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
- Brief overview of the current state of artificial intelligence
- Importance of staying updated on AI trends and developments

Key Points:
1. Latest Trends in Artificial Intelligence:
- Advancements in machine learning and deep learning
- Integration of AI in various industries such as healthcare, finance, and automotive
- Ethical considerations and regulations in AI development

2. Key Players in the Artificial Intellige

- Display the results of your execution as markdown in the notebook.

In [13]:
# save the general workflow reseult as a markdown file and print its content
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has rapidly become a transformative force in our world, revolutionizing industries and reshaping the way we live and work. Staying informed on the latest AI trends and developments is crucial in understanding the future implications of this cutting-edge technology.

## Latest Trends in Artificial Intelligence
Advancements in machine learning and deep learning have propelled AI to new heights, enabling machines to learn from data and make decisions with minimal human intervention. This has led to the integration of AI in various sectors such as healthcare, finance, and automotive, where AI-powered solutions are optimizing processes and driving innovation. However, with great power comes great responsibility, as ethical considerations and regulations in AI development are paramount to ensure the responsible use of this powerful technology.

## Key Players in the Artificial Intelligence Industry
Leading tech giants like Google, Amazon, and Microsoft are at the forefront of AI research and development, investing heavily in cutting-edge technologies and solutions. Additionally, emerging startups are disrupting the AI landscape with innovative approaches and solutions that are challenging the status quo. Collaboration between academia and industry is pushing the boundaries of AI, fostering a dynamic ecosystem of research and innovation.

## Noteworthy News in Artificial Intelligence
Breakthroughs in natural language processing and computer vision have opened up new possibilities for AI applications, from virtual assistants to autonomous vehicles. AI is also being leveraged to combat climate change and improve sustainability, with algorithms and models driving impactful solutions. However, the impact of AI on job markets and workforce automation raises important questions about the future of work and the need for reskilling and upskilling in the age of AI.

## Ethical Considerations in AI Development
As AI becomes more pervasive in our society, ethical considerations in AI development are becoming increasingly important. From bias in algorithms to data privacy concerns, navigating the ethical dilemmas of AI requires a thoughtful and proactive approach. Establishing guidelines and frameworks for responsible AI development is essential to ensure that AI benefits society as a whole.

## Future Implications of AI
The future of artificial intelligence holds immense potential for both opportunities and challenges. As AI continues to evolve and expand its capabilities, it will reshape industries, redefine jobs, and transform the way we interact with technology. Staying informed and engaged with AI developments through reputable sources is key to navigating the complex landscape of AI and harnessing its benefits for a better future.

## Call to Action
In conclusion, the future of artificial intelligence is filled with promise and potential, but it also presents ethical, social, and economic challenges that must be addressed. By staying informed and engaged with AI developments, we can shape the future of AI in a responsible and sustainable manner. Let us embrace the opportunities that AI brings while also being mindful of the risks and implications, ensuring a future where AI works for the betterment of society.

Remember to stay updated on Artificial Intelligence trends, explore the key players in the AI industry, and follow the latest news on AI developments to be at the forefront of this technological revolution.

## Additional Execution Example - Try it yourself!

- Pass in a topic of your choice and see what the agents come up with!

In [14]:
# topic = "YOUR TOPIC HERE"
# result = crew.kickoff(inputs={"topic": topic})

In [15]:
topic = "Quantum mechanics"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Quantum mechanics.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: Unraveling the Mysteries of Quantum Mechanics

Introduction:
- Brief explanation of quantum mechanics and its significance in modern physics
- Overview of the latest trends and advancements in quantum mechanics research
- Mention key players in the field and recent noteworthy news articles

Target Audience:
- Science enthusiasts, students, and professionals in the field of physics
- Those interested in cutting-edge scientific discoveries and theories
- Individuals looking to expand th

In [16]:
Markdown(result)

# Unraveling the Mysteries of Quantum Mechanics

## Introduction to Quantum Mechanics

Quantum mechanics, a fundamental theory in physics, has revolutionized our understanding of the universe at its most fundamental levels. Developed in the early 20th century, this theory describes the behavior of particles at the atomic and subatomic levels, challenging our classical notions of reality. Quantum mechanics plays a crucial role in modern physics, with applications in areas such as quantum computing, cryptography, and information theory.

Recent years have seen significant breakthroughs in quantum mechanics research, particularly in the fields of quantum computing, entanglement, and superposition. Quantum computing, in particular, holds the promise of exponentially faster computation and has the potential to revolutionize industries ranging from healthcare to finance. Entanglement, a phenomenon where particles become interconnected regardless of distance, continues to baffle scientists and push the boundaries of our understanding of the quantum world. Superposition, the concept that particles can exist in multiple states simultaneously, opens up possibilities for new technologies and innovations.

## Key Players and Noteworthy News

Prominent scientists in the field of quantum mechanics, such as Niels Bohr, Max Planck, and Richard Feynman, have made significant contributions to the development of the theory. Their work continues to inspire new generations of researchers and students. Recent news articles on quantum mechanics have highlighted advancements in quantum teleportation, quantum encryption, and the quest for a unified theory of quantum gravity.

## Call to Action

As we continue to unravel the mysteries of quantum mechanics, it is essential to stay informed and engage with the latest developments in the field. I encourage readers to explore further resources on quantum mechanics, including scientific journals, research institutions, and books on the subject. Join the conversation, share your thoughts, and be a part of the exciting journey into the fascinating world of quantum physics.

In conclusion, quantum mechanics remains a captivating and ever-evolving field of study, with endless possibilities for discovery and innovation. By staying curious and informed, we can all contribute to the advancement of this groundbreaking theory and shape the future of science.

Thank you for joining me on this exploration of quantum mechanics, and I look forward to continuing the conversation with you.

## Downloading the markdown file

In [17]:
# !pip install markdown

In [18]:
import markdown

In [19]:
markdown_content = Markdown(result)
file_name = "quantum_mechanics.md"

with open(file_name, 'w', encoding='utf-8') as file:
    file.write(str(markdown_content))


In [20]:
with open(file_name, 'r') as file:
    content = file.read()
print(content)


<IPython.core.display.Markdown object>


<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

# Conclusion

This notebook demonstrates how to automate content creation using a multi-agent system, providing a robust framework for generating high-quality written content on various topics.